In [1]:
import dlib
from glob import glob 
import cv2
import numpy as np
import os

# load the face detector, landmark predictor, and face recognition model
face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor("C:\\Users\\annan\\Documents\\Study\\Konda\\AIDI Course\\CapstoneProjects\\CapstoneProject-2\\images\\shape_predictor_68_face_landmarks.dat")
face_encoder = dlib.face_recognition_model_v1("C:\\Users\\annan\Documents\\Study\\Konda\\AIDI Course\\CapstoneProjects\\CapstoneProject-2\\images\\dlib_face_recognition_resnet_model_v1.dat")


In [2]:
VALID_EXTENSIONS = ['.png', '.jpg', '.jpeg']

def get_image_paths(root_dir, class_names):
    """ grab the paths to the images in our dataset"""
    image_paths = []
    for class_name in class_names:
        # grab the paths to the files in the current class directory
        class_dir = os.path.sep.join([root_dir, class_name])
        class_file_paths = glob(os.path.sep.join([class_dir, '*.*']))

        # loop over the file paths in the current class directory
        for file_path in class_file_paths:
            # extract the file extension of the current file
            ext = os.path.splitext(file_path)[1]

            # if the file extension is not in the valid extensions list, ignore the file
            if ext.lower() not in VALID_EXTENSIONS:
                print("Skipping file: {}".format(file_path))
                continue

            # add the path to the current image to the list of image paths
            image_paths.append(file_path)

    return image_paths

In [3]:
def face_rects(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # detect faces in the grayscale image
    rects = face_detector(gray, 1)
    # return the bounding boxes
    return rects

In [4]:
def face_landmarks(image):
    return [shape_predictor(image, face_rect) for face_rect in face_rects(image)]

In [5]:
def face_encodings(image):
    # compute the facial embeddings for each face 
    # in the input image. the `compute_face_descriptor` 
    # function returns a 128-d vector that describes the face in an image
    return [np.array(face_encoder.compute_face_descriptor(image, face_landmark)) 
            for face_landmark in face_landmarks(image)]

In [6]:
def nb_of_matches(known_encodings, unknown_encoding):
    # compute the Euclidean distance between the current face encoding 
    # and all the face encodings in the database
    distances = np.linalg.norm(known_encodings - unknown_encoding, axis=1)
    # keep only the distances that are less than the threshold
    small_distances = distances <= 0.6
    # return the number of matches
    return sum(small_distances)

In [9]:
import pickle
import cv2
import os


root_dir = "C:\\Users\\annan\\Documents\\Study\\Konda\\AIDI Course\\CapstoneProjects\\CapstoneProject-2\\images"
class_names = os.listdir(root_dir)

# get the paths to the images
image_paths = get_image_paths(root_dir, class_names)
# initialize a dictionary to store the name of each person and the corresponding encodings
name_encondings_dict = {}
nb_current_image = 1
# now we can loop over the image paths, locate the faces, and encode them
for image_path in image_paths:
    print(f"Image processed {nb_current_image}/{len(image_paths)}")
    # load the image
    image = cv2.imread(image_path)
    # get the face embeddings
    encodings = face_encodings(image)
    # get the name from the image path
    name = image_path.split(os.path.sep)[-2]
    # get the encodings for the current name
    e = name_encondings_dict.get(name, [])
    # update the list of encodings for the current name
    e.extend(encodings)
    # update the list of encodings for the current name
    name_encondings_dict[name] = e
    nb_current_image += 1
with open("C:\\Users\\annan\\Documents\\Study\\Konda\\AIDI Course\\CapstoneProjects\\CapstoneProject-2\\pickleFile\\encodings.pickle", "wb") as f:
    pickle.dump(name_encondings_dict, f)

Image processed 1/6
Image processed 2/6
Image processed 3/6
Image processed 4/6
Image processed 5/6
Image processed 6/6


In [9]:
import pickle
import cv2

# load the encodings + names dictionary
with open("C:\\Users\\annan\\Documents\\Study\\Konda\\AIDI Course\\CapstoneProjects\\CapstoneProject-2\\pickleFile\\encodings.pickle", "rb") as f:
    name_encodings_dict = pickle.load(f)
    
# import  tkinter as tk
# from tkinter import filedialog
# root = tk.Tk()
# root.withdraw()
# file_path = filedialog.askopenfilename()
# file_path='C:\\Users\\annan\\Documents\\Study\\Konda\\AIDI Course\\CapstoneProjects\\CapstoneProject-2\\testImaage\\WIN_20230615_13_30_39_Pro.jpg'
# file_path='C:\\Users\\annan\\Documents\\Study\\Konda\\AIDI Course\\CapstoneProjects\\CapstoneProject-2\\testImaage\\WIN_20230612_04_50_31_Pro.jpg'
file_path='C:\\Users\\annan\\Documents\\Study\\Konda\\AIDI Course\\CapstoneProjects\\CapstoneProject-2\\testImaage\\WhatsApp Image 2023-06-15 at 14.04.26.jpg'
# load the input image
image = cv2.imread(file_path)
# get the 128-d face embeddings for each face in the input image
encodings = face_encodings(image)
# this list will contain the names of each face detected in the image
names = []
# ...

# loop over the encodings
for encoding in encodings:
    # initialize a dictionary to store the name of the 
    # person and the number of times it was matched
    counts = {}
    # loop over the known encodings
    for (name, encodings) in name_encodings_dict.items():
        # compute the number of matches between the current encoding and the encodings 
        # of the known faces and store the number of matches in the dictionary
        counts[name] = nb_of_matches(encodings, encoding)
    # check if all the number of matches are equal to 0
    # if there is no match for any name, then we set the name to "Unknown"
    if all(count == 0 for count in counts.values()):
        name = "Unknown"
    # otherwise, we get the name with the highest number of matches
    else:
        name = max(counts, key=counts.get)
    
    # add the name to the list of names
    names.append(name)
for rect, name in zip(face_rects(image), names):
    # get the bounding box for each face using the `rect` variable
    x1, y1, x2, y2 = rect.left(), rect.top(), rect.right(), rect.bottom()
    # draw the bounding box of the face along with the name of the person
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image, name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
    # show the output image
    cv2.imshow("image", image)
    cv2.waitKey(0)